In [1]:
# !pip install pycaret[full]

In [ ]:
# !pip install geopandas

In [14]:
import pandas as pd
import numpy as np
import geopandas as gpd
from tqdm.auto import tqdm


In [15]:
def get_coordinates(df,column_name):
    lat = []
    lon = []
    df = df.copy()
    for geodata in tqdm(df[column_name]):
        centroid = gpd.read_file(geodata,driver='GeoJSON').to_crs(crs=3857).centroid
        lat.append(centroid.x)
        lon.append(centroid.y)
    df.drop(column_name,axis=1,inplace=True)
    df['lat'] = np.array(lat)
    df['long'] = np.array(lon)
    return df

In [16]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [19]:
train = get_coordinates(train,'.geo')
test = get_coordinates(test,'.geo')

  0%|          | 0/4830 [00:00<?, ?it/s]

  0%|          | 0/2071 [00:00<?, ?it/s]

In [ ]:
from pycaret.classification  import *

s = setup(train,
          target='crop',
          ignore_features = ['id'],
          session_id = 133,)

In [21]:
model = create_model('lightgbm')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9793,0.9992,0.9791,0.9796,0.9793,0.9758,0.9759
1,0.9763,0.9991,0.9774,0.9764,0.9762,0.9724,0.9724
2,0.9615,0.9976,0.9624,0.9625,0.9619,0.9551,0.9552
3,0.9734,0.9992,0.9743,0.9739,0.9734,0.9689,0.9690
4,0.9734,0.9989,0.9734,0.9754,0.9735,0.9689,0.9692
5,0.9793,0.9997,0.9795,0.9793,0.9793,0.9758,0.9758
6,0.9615,0.9975,0.9606,0.9638,0.9614,0.9551,0.9555
7,0.9645,0.9990,0.9657,0.9649,0.9642,0.9586,0.9588
8,0.9556,0.9990,0.9559,0.9574,0.9559,0.9482,0.9484


INFO:logs:create_model_container: 1
INFO:logs:master_model_container: 1
INFO:logs:display_container: 2
INFO:logs:LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=133, reg_alpha=0.0, reg_lambda=0.0, silent='warn',
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)
INFO:logs:create_model() succesfully completed......................................


In [ ]:
res = predict_model(model,test)

In [23]:
res.rename(columns={
    'Label':'crop'  
},inplace=True)
res[['id','crop']].to_csv('pred.csv',index=False)